# Libs

In [ ]:
!pip install mistralai -U -q
!pip install nest_asyncio -U -q
!pip install rich -U -q
!pip install python-dotenv -U -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Async Notebook

In [3]:
from rich import print
import nest_asyncio


nest_asyncio.apply()

# Env

In [4]:
import os
from dotenv import load_dotenv 

# default directory for .env file is the current directory
# if you set .env in different directory, put the directory address load_dotenv("directory_of_.env)
load_dotenv()

True

# Text


In [ ]:
from pydantic_ai import Agent
from pydantic import BaseModel
from pydantic_ai.models.openai import MistralModel

In [ ]:
model = MistralModel(
    model_name='mistral-large-latest', 
    api_key=os.environ.get('MISTRAL_API_KEY')
)
agent = Agent(model)

result = agent.run_sync('Could you give a random city? in france')
print(result.data)
print(result.cost())

/var/folders/dj/wy15spl11jq1vgrmw7rltbn80000gn/T/ipykernel_98958/489037473.py:4: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  result = agent.run_sync('Could you give a random city? in france')


Sure! How about Bordeaux? It's a beautiful city in southwestern France known for its wine, architecture, and rich 
history.

Cost(request_tokens=13, response_tokens=29, total_tokens=42, details=None)

In [14]:
model = MistralModel(
    model_name='mistral-large-latest', 
    api_key=os.environ.get('MISTRAL_API_KEY')
)
agent = Agent(model)

result = agent.run_stream('Could you give a random city? in france')

async with result as result:
    async for profile in result.stream():
        print(profile)
        

print("Cost:", result.cost())
print("Final", await result.get_data())

Sure! How about the city

Sure! How about the city of Lyon? It

Sure! How about the city of Lyon? It's known for its rich history

Sure! How about the city of Lyon? It's known for its rich history, vibrant culture, and

Sure! How about the city of Lyon? It's known for its rich history, vibrant culture, and delicious cuisine.

Sure! How about the city of Lyon? It's known for its rich history, vibrant culture, and delicious cuisine.

Cost:
Cost(request_tokens=13, response_tokens=28, total_tokens=41, details=None)

Final

# Structured 

In [ ]:

class CityLocation(BaseModel):
    city: str
    

model = MistralModel(model_name='mistral-large-latest', api_key=os.environ.get('MISTRAL_API_KEY'))
agent = Agent(model,result_type=CityLocation)

result = agent.run_sync('Could you give a random city? in france')
print(result.data)
print(result.cost())

CityLocation(city='Paris')

Cost(request_tokens=88, response_tokens=22, total_tokens=110, details=None)

# Basic call

In [5]:
from pydantic_ai import Agent
from pydantic import BaseModel

from pydantic_ai.models.openai import MistralModel

class CityLocation(BaseModel):
    city: str
    

model = MistralModel(model_name='mistral-large-latest', api_key=os.environ.get('MISTRAL_API_KEY'))
agent = Agent(model,result_type=CityLocation)

result = agent.run_sync('Could you give a random city? in france')
print(result.data)
#> city='London' country='United Kingdom'
print(result.cost())
#> Cost(request_tokens=56, response_tokens=8, total_tokens=64, details=None)

/var/folders/dj/wy15spl11jq1vgrmw7rltbn80000gn/T/ipykernel_98005/3267781817.py:13: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  result = agent.run_sync('Could you give a random city? in france')


CityLocation(city='Paris')

Cost(request_tokens=88, response_tokens=22, total_tokens=110, details=None)

In [ ]:
# test


In [ ]:
agent = Agent(model,result_type=CityLocation)
async with agent.run_stream('Could you give a random city? in france') as result:
    async for profile in result.stream():
        print(profile)


# With function

In [ ]:
from dataclasses import dataclass
from pydantic_ai import Agent, RunContext
from typing import Literal
# Define the dependencies class
@dataclass
class Deps:
    winning_number: int


model = MistralModel(model_name='mistral-large-latest', api_key=os.environ.get('MISTRAL_API_KEY'))

# Create the agent with proper typing
roulette_agent = Agent(
    model,
    deps_type=Deps,
    retries=3,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to determine if the '
        'customer has won based on the number they bet on.'
    ),
)


@roulette_agent.tool
async def roulette_wheel(
    ctx: RunContext[Deps], square: int
) -> Literal['winner', 'loser']:
    """Check if the bet square is a winner.

    Args:
        ctx: The context containing the winning number.
        square: The number the player bet on.
    """
    return 'winner' if square == ctx.deps.winning_number else 'loser'


# Set up dependencies
winning_number = 18
deps = Deps(winning_number=winning_number)

    # Run some example bets using streaming
response = roulette_agent.run_sync(
    'Put my money on square eighteen', deps=deps
) 
print('Response', response)
print('Bet on 18:', response.data)



Response
RunResult(
    _all_messages=[
        SystemPrompt(
            content='Use the `roulette_wheel` function to determine if the customer has won based on the number 
they bet on.',
            role='system'
        ),
        UserPrompt(
            content='Put my money on square eighteen',
            timestamp=datetime.datetime(2024, 12, 8, 15, 32, 32, 40276, tzinfo=datetime.timezone.utc),
            role='user'
        ),
        ModelStructuredResponse(
            calls=[
                ToolCall(
                    tool_name='roulette_wheel',
                    args=ArgsJson(args_json='{"square": 18}'),
                    tool_id='sRuX1y7NH'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 8, 15, 32, 32, tzinfo=datetime.timezone.utc),
            role='model-structured-response'
        ),
        ToolReturn(
            tool_name='roulette_wheel',
            content='winner',
            tool_id='sRuX1y7NH',
            timestamp=datetime.datetime(2024, 12, 8, 15, 32, 33, 59693, tzinfo=datetime.timezone.utc),
            role='tool-return'
        ),
        ModelStructuredResponse(
            calls=[
                ToolCall(
                    tool_name='final_result',
                    args=ArgsJson(args_json='{"response": true}'),
                    tool_id='t6v8E07ut'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 8, 15, 32, 33, tzinfo=datetime.timezone.utc),
            role='model-structured-response'
        )
    ],
    _new_message_index=1,
    data=True,
    _cost=Cost(request_tokens=452, response_tokens=41, total_tokens=493, details=None)
)

Bet on 18: True

In [ ]:

async with roulette_agent.run_stream(
    'I bet five is the winner', deps=deps
) as response:
    result = await response.get_data()
    print('Bet on 5:', result)